In [20]:
import collections
import pandas as pd
import datetime
import numpy as np
import json
import codecs

In [21]:
data_fr = pd.read_csv('../data/data_fr.csv', sep=';', usecols=np.arange(2,35))
corres_ICD10 = pd.read_csv('../data/corres_ICD10.csv', sep=';',usecols=[1,2,3,4,5])
corres_geo = pd.read_csv('../data/corres_geo.csv', sep=';', usecols=[1,2,3,4])

In [22]:
corres_geo.head()

,niveau_0,niveau_1,niveau_2,region
0,FR,FR,FR,France
1,FR,FR1,FR1,ile de France
2,FR,FR1,FR10,ile de France
3,FR,FR2,FR2,Bassin Parisien (NUTS 2013)
4,FR,FR2,FR21,Champagne-Ardenne (NUTS 2013)


In [23]:
corres_ICD10.head(10)

,niveau_0,niveau_1,niveau_2,niveau_3,maladies
0,A-R_V-Y,A-R_V-Y,A-R_V-Y,A-R_V-Y,Toutes causes de mortalite (A00-Y89) sauf S00-T98
1,A-R_V-Y,A_U,A_U,A_U,Causes interne de morbidite et mortalite
2,A-R_V-Y,A_U,A_B,A_B,Certaines maladies infectieuses et parasitaire...
3,A-R_V-Y,A_U,A_B,A15-A19_B90,Tuberculose
4,A-R_V-Y,A_U,A_B,B15-B19_B942,Hepatite virale et sequelles d'hepatite virale
5,A-R_V-Y,A_U,A_B,B20-B24,Maladies dues au virus de l'immunodeficience h...
6,A-R_V-Y,A_U,A_B,A_B_OTH,Autres certaines maladies infectieuses et para...
7,A-R_V-Y,A_U,C_D,C00-D48,Tumeurs
8,A-R_V-Y,A_U,C_D-C,C,Tumeurs malignes (C00-C97)
9,A-R_V-Y,A_U,C_D-C,C00-C14,"Tumeur maligne de la levre, de la cavite bucca..."


In [24]:
# Nettoyage des données:
for col in data_fr.columns[11:]:
    data_fr[col] = data_fr[col].str.strip().replace({":":"NaN", ": z":"NaN"}).astype('float64')

In [27]:
data_fr = data_fr.drop(['icd10_niv_3','geo_niv_2'], axis=1)

In [28]:
data_fr.head()

,sex,age,geo_niv_0,geo_niv_1,region,icd10_niv_0,icd10_niv_1,icd10_niv_2,maladies,2015,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994
0,F,TOTAL,FR,FR,France,A-R_V-Y,A-R_V-Y,A-R_V-Y,Toutes causes de mortalite (A00-Y89) sauf S00-T98,864.04,...,868.2,839.1,835.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F,TOTAL,FR,FR1,ile de France,A-R_V-Y,A-R_V-Y,A-R_V-Y,Toutes causes de mortalite (A00-Y89) sauf S00-T98,602.05,...,684.3,640.7,649.4,649.2,660.7,670.5,671.1,681.2,686.3,669.9
2,F,TOTAL,FR,FR2,Champagne-Ardenne (NUTS 2013),A-R_V-Y,A-R_V-Y,A-R_V-Y,Toutes causes de mortalite (A00-Y89) sauf S00-T98,971.97,...,898.5,866.6,870.9,877.6,900.1,858.5,861.1,874.0,881.0,842.5
3,F,TOTAL,FR,FR2,Picardie (NUTS 2013),A-R_V-Y,A-R_V-Y,A-R_V-Y,Toutes causes de mortalite (A00-Y89) sauf S00-T98,893.97,...,880.7,844.1,830.9,829.0,839.5,828.4,838.8,844.8,834.7,794.2
4,F,TOTAL,FR,FR2,Haute-Normandie (NUTS 2013),A-R_V-Y,A-R_V-Y,A-R_V-Y,Toutes causes de mortalite (A00-Y89) sauf S00-T98,888.27,...,867.2,827.0,833.0,820.2,814.5,830.9,811.7,844.8,833.0,777.1


In [29]:
data_fr.columns

Index(['sex', 'age', 'geo_niv_0', 'geo_niv_1', 'region', 'icd10_niv_0',
       'icd10_niv_1', 'icd10_niv_2', 'maladies', '2015', '2014', '2013',
       '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004',
       '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995',
       '1994'],
      dtype='object')

In [17]:
df = data_fr.pivot_table(index=data_fr.columns[2:11].to_list(),
                   values=data_fr.columns[11:].to_list(),
                   columns=data_fr.columns[:2].to_list())
df

AttributeError: 'Index' object has no attribute 'to_list'

In [8]:
df.to_json('test.json')

In [13]:
# ============ Dictionnaires imbriqués ===========
# ================================================

# def nested_dict():
#     return collections.defaultdict(nested_dict)
# result = nested_dict()

# for row in df.itertuples():
#     result[row.Index[0]][row.Index[1]][row.Index[2]][row.Index[3]][row.Index[4]][row.Index[5]] = row._1
    
# out = json.dumps(result)

# with open('test.json', 'w') as f:
#     f.write(out)